In [ ]:
import hylite
from hylite import io, HyLibrary
import numpy as np

import matplotlib.pyplot as plt

In [ ]:
%matplotlib inline

Hyperspectral data that is collected outdoors (e.g. data acquired from a satellite, UAV or tripod) must be carefully corrected for the spectral signature of the various light-sources (e.g., sunlight, skylight) involved before accurate reflectance spectra can be estimated.

*hylite* includes a variety of methods for doing this, from the commonly applied empirical line correction to more complicated approaches using multiple light sources and bidirectional reflectance distribution functions (BRDFs). The theory behind these topographic and atmospheric corrections (hereafter termed 'illumination corrections') are described in some detail [here](https://ieeexplore.ieee.org/stamp/stamp.jsp?arnumber=9507524) and [here](https://www.mdpi.com/2072-4292/14/1/5), but in the following notebook we give a more practical explanation of their application.

### 1. Forward modelling

To explain the logic behind this illumination correction approach (and to avoid uploading yet more data ㋛ ), we will start by taking an already corrected hypercloud and then add topographic and atmospheric effects such that it simulates a radiance dataset as might be measured by a field spectrometer. Then, in the following section, we will correct it to arrive back at reflectance.

First, lets load the hypercloud and put it and the relevant geometric information into HyImage instances. Note that in a real situation these geometric attributes are easilly accessible via the `HyScene` class.

In [ ]:
# load hypercloud dataset
cloud = io.load( 'test_data/hypercloud.hdr' )
cloud.decompress() # this was compressed from float to integer to save space; so we need to convert it back
cam = cloud.header.get_camera(0) # viewing position (see Notebook 02 to derive this from field data)

# render to 2D images
refl = cloud.render(cam, bands=(0,-1))
xyz = cloud.render(cam, 'xyz')
normals = cloud.render(cam, 'klm')
for img in [refl,xyz,normals]: # fill in small gaps between some points
    img.fill_holes() 
    img.set_as_nan(0)

# compute (normalised) view vectors
view = xyz.copy(data=True)
view.data -= cam.pos
view.data /= np.linalg.norm(view.data, axis=-1)[...,None]

# compute pixel depths
depth = xyz.copy(data=True)
depth.data -= cam.pos
depth.data = np.linalg.norm(depth.data, axis=-1)[...,None]

# plot everything
fig,ax=plt.subplots(1,5,figsize=(20,6))
refl.quick_plot(hylite.SWIR,ax=ax[0])
xyz.quick_plot((0,1,2),ax=ax[1],tscale=True, vmin=5, vmax=95)
normals.quick_plot((0,1,2),ax=ax[2])
view.quick_plot((0,1,2),ax=ax[3], tscale=True, vmin=5, vmax=95)
depth.quick_plot(0,cmap='coolwarm',ax=ax[4])
for a,t in zip(ax,['a. SWIR','b. xyz','c. normals','d. view vector','e. depth']):
    a.set_title(t)

fig.show()
fig.tight_layout()

We will also add some virtual calibration panels to our scene, as would be done if acquiring field data. These will be used later to estimate and correct for illumination effects.

In [ ]:
# compute panel orientation
n = np.nanmean( normals.data, axis=(0,1) ) # give this the average orientation of the cliff
n = n / np.linalg.norm(n)

# orientation of shaded panel
ns = -np.array([1,-1.,-1.0]) # oriented so that it is not directly lit
ns = ns / np.linalg.norm(ns)

# depth of panels (average of outcrop)
dd = np.nanmedian(depth.data)

# add fake calibration panels for next sections
pad = 5
for yy,n,r,z in zip([(50,100),(100,150),(150,200)],[n,n,ns], [0.25, 0.5, 0.99 ], [dd,dd, 30. ] ):
    h = int( (yy[1]-yy[0]) * 0.45 ) - pad
    for y,_z in zip([yy[0]+pad,yy[1]-h-pad],[z,30.]): # split into two panels; when near the target, one near the sensor
        refl.data[460:490,y:(y+h),:] = r # add panel to reflectance data
        normals.data[460:490,y:(y+h),:] = n # add panel orientation
        depth.data[460:490,y:(y+h),:] = _z # add far panel depth
        view.data[460:490,y:(y+h),:] =  view.data[440,100,:]

Although reality is far more complex, we will assume that our scene is lit by two light sources only; skylight and sunlight. Skylight is comes (we assume uniformly) from the entire upper-hemisphere (sky), while sunlight comes from an approximate point-source (the sun). The amount of reflected light will depend on the material reflectance factor at the relevant wavelength, and the amount and orientation of downwelling light:

$$[ Eq. 1 ] \space radiance = reflectance \times ( skylight \times \alpha + sunlight \times \beta ) + path\_radiance$$ 

$\alpha$ and $\beta$ are values that capture the effect of topography/geometry on the reflection, and depending on the model used, account for factors such as the incidence angle of the light source and roughness of the surface.

The distribution of skylight across the scene depends on the amount of sky visible from each point, termed the 'sky view factor'. In real situations this should be calculated based on the scene geometry (e.g. using [CloudCompare](https://www.cloudcompare.org/doc/wiki/index.php?title=ShadeVis_(plugin))), but for this simple example we will use a simple approximation based only on the surface orientation (normal vector) to derive a value for $\alpha$:

In [ ]:
from hylite.correct.illumination import estimate_skyview
skyview = xyz.copy(data=False)
skyview.data = estimate_skyview( normals.data )[...,None]
skyview.fill_holes()
skyview.set_as_nan(0)

fig,ax = skyview.quick_plot(0,cmap='gray', vmin=0, vmax=1., figsize=(12,7))
ax.set_title("Estimated skyview factor")
fig.colorbar(ax.cbar, orientation='vertical', shrink=0.5) # add colorbar for reference
fig.tight_layout()
fig.show()

The amount of sunlight that reflects to the hyperspectral sensor will depend on the incidence angle of the sunlight and the reflection distribution function. *hylite* currently has two different reflection distribution functions implemented: Lambert and Oren-Nayar. Both of these depend on the illumination direction, so we need to define the illumination direction. This can be calculated for any time and position thanks to the *astral* package.

In [ ]:
from hylite.correct.illumination import estimate_sun_vec
sunvec,az,el = estimate_sun_vec(37.7, -6.6, ("19/04/2019 12:28","%d/%m/%Y %H:%M", 'Europe/Madrid') )
print("Sun orientation is %d->%d %s" % (el, az, sunvec) )

Lambert's law assumes a perfectly smooth surface, such that the light reflected in any direction will be equal to the cosine of the incidence angle. This is a poor approximation for most natural materials (e.g. rocks, plants), but is useful for calibration panels, which are engineered to be as close to lambertian as possible (more on this later). However, for completeness, we can calculate $\beta$ using a lambertian model as follows:

In [ ]:
from hylite.correct.illumination import calcLambert
L = xyz.copy(data=False)
L.data = calcLambert( normals.data, sunvec )

fig,ax = L.quick_plot(0,cmap='gray', vmin=0, vmax=1., figsize=(12,7))
ax.set_title("Lambertian reflectance factor")
fig.colorbar(ax.cbar, orientation='vertical', shrink=0.5) # add colorbar for reference
fig.tight_layout()
fig.show()

The Oren-Nayar reflection model provides a better approximation for rocks and other materials with a rough surface. It assumes any point on a surface can be approximated by an infinite number of lambertian facets with a normally distribution orientation, defined by the roughness factor $\sigma$. This roughness results in more reflected light than would be otherwise predicted by a Lambertian model. Also, unlike the Lambertian model, orientation of these facets relative to the viewing direction thus also becomes important, making the Oren-Nayar model a simple BRDF.

In [ ]:
from hylite.correct.illumination import calcOrenNayar
ON = xyz.copy(data=False)
ON.data = calcOrenNayar( normals.data, view.data, sunvec, roughness=0.2 )

fig,ax = ON.quick_plot(0,cmap='gray', vmin=0, vmax=1., figsize=(12,7))
ax.set_title("Oren-Nayar reflectance factor")
fig.colorbar(ax.cbar, orientation='vertical', shrink=0.5) # add colorbar for reference
fig.tight_layout()
fig.show()

Assuming a known spectra for sunlight and skylight, and a constant path radiance, these estimated sky-view factors and Oren-Nayar reflectance coefficients can be combined to create a synthetic radiance dataset. For convenience, we have saved some realistic sun and skylight spectra in a HyLibrary in the example dataset, which we now load.

In [ ]:
I = io.load('test_data/illumination.hdr')
fig,ax = I.quick_plot(color=['green','blue','gold'], figsize=(8,4))
fig.show()

In [ ]:
# multiply path radiance by per-pixel depth to simulate increasing path radiance with depth
P = depth.copy(data=False)
P.data = depth.data * I['path'].data / 1000. # convert depths from m to km
P.set_wavelengths( I.get_wavelengths() )

We can then generate a simulated radiance dataset using Eq. 1. For convenience, this has already been implemented *hylite* with the `IlluModel` class.

In [ ]:
from hylite.correct.illumination import IlluModel
I = I.resample(refl.get_wavelengths(),vb=False) # resample onto same wavelength array as our reflectance data
P = P.resample(refl.get_wavelengths(),vb=False)

model = IlluModel( I = I['sun'].data.ravel(), # sunlight spectra
                   S = I['sky'].data.ravel(), # skylight spectra
                   P = P, # path radiance
                   skv = skyview, # skyview factors
                   rf = ON, # reflectance factors
                   oc = None, # for this simple example we assume no projective shadows
                 )

# evaluate radiance and plot it
radiance = refl.copy(data = False )
radiance.data = model.getRadiance( refl)

In [ ]:
fig,ax = plt.subplots(1,2, figsize=(18,6), gridspec_kw={'width_ratios': [2, 1]})
samples = [(200,200), (400,100), (475,65), (475,115), (475,165) ]
cols = ['red', 'orange', 'green', 'gold', 'blue' ]
ax[0].set_title("Radiance image")
radiance.quick_plot(hylite.SWIR, ax=ax[0],vmin=0,vmax=99, samples=samples, ps=15,pc=cols, ticks=True)
ax[1].set_title("Radiance spectra")
radiance.plot_spectra(ax=ax[1], indices=samples, colours=cols)
ax[1].set_ylabel(r"Radiance ($\frac{W}{nm \cdot m^{2} \cdot sr }$)")
fig.tight_layout()
fig.show()

Note the number of atmospheric absorbtion features that now exist in the radiance spectra - these highlight why it is so important to convert to reflectance before making spectral interpretations or classifications.

### 2. Calibration panels

Finally, to complete our simulated field dataset, we will extract the radiance spectra from three calibration panels. Two of these are placed in the scene with reflectance 5% and 50%, and the third is placed next to the sensor (no path radiance) and shaded to block any downwelling sunlight. These are placed in a `Panel` class to facilitate processing in the next section. 

-----------
*Note that `Panel` instances can be easily created for real data by picking the corners of the imaged calibration panels. See `Panel.__init__(...)` for details. These corners can also be used for estimating panel orientation (assuming it is square and the viewing geometry is known), 
and hence other important properties (skyview factor, incidence angle) -  see `Panel.get_normal(...)`* 

----

In [ ]:
from hylite.correct.illumination import Panel
from hylite.reference.spectra import Target

# evaluate panel radiance using a lambert reflectance model
pr = IlluModel( I = I['sun'].data.ravel(), S = I['sky'].data.ravel(),
                   P = P, skv = skyview, rf = L, oc = None ).getRadiance( refl )


R25_far = Panel( Target(refl.get_wavelengths(), np.ones(refl.band_count())*0.25, name='R25'), # define panel material
            pr[475,65,:], wavelengths=radiance.get_wavelengths() )
R25_close = Panel( Target(refl.get_wavelengths(), np.ones(refl.band_count())*0.25, name='R25'), # define panel material
            pr[475,85,:], wavelengths=radiance.get_wavelengths() )

R50_far = Panel( Target(refl.get_wavelengths(), np.ones(refl.band_count())*0.5, name='R50'), # define panel material
            pr[475,115,:], wavelengths=radiance.get_wavelengths() )
R50_close = Panel( Target(refl.get_wavelengths(), np.ones(refl.band_count())*0.5, name='R50'), # define panel material
            pr[475,135,:], wavelengths=radiance.get_wavelengths() )

R99_shade = Panel( Target(refl.get_wavelengths(), np.ones(refl.band_count())*0.99, name='R99'), # define panel material
            pr[475,165,:], wavelengths=radiance.get_wavelengths() )

# add skyview and reflectance factor attributes. These can be calculated for real panels (with known orientations).
for p in [R25_far,R25_close,R50_far,R50_close]:
    p.skyview = skyview.data[475,85,0]
    p.alpha = L.data[475,85,0]
R99_shade.skyview = skyview.data[475,135,0] # skyview panel has different skyview
    

In [ ]:
fig,ax = plt.subplots(figsize=(10,5))
for R,c in zip( [R25_far,R50_far,R99_shade],['darkgray','lightgray','blue']):
    ax.plot(R.get_wavelengths(), R.get_mean_radiance(), color=c)
ax.set_ylabel('Measured Radiance')
fig.show()

Note that as we are only dealing with SWIR spectra the skylight illumination is very low; our calibration spectra would look very different if we were using VNIR data.

### 3. Empirical line correction

The simplest topographic correction is the empirical line correction, or ELC. This assumes a constant illumination spectra across the scene, which is reconstructed by fitting a line to the calibration panels and used to convert the radiance to reflectance. This can be done in *hylite* using the `ELC` class, providing one or more calibration panels are available.

In [ ]:
from hylite.correct.illumination import ELC
elc = ELC([R25_far, R50_far]) # N.B we exclude the shaded panel from this as it would screw everything up
fig,ax = elc.quick_plot(thresh=100, figsize=(7,5))
fig.show()

In [ ]:
reflectance = radiance.copy(data=True) # create copy of data
_ = elc.apply(reflectance) # apply ELC

fig,ax = plt.subplots(1,2, figsize=(18,5))
ax[0].set_title("ELC corrected reflectance")
reflectance.plot_spectra(ax=ax[0], indices=samples[:2], colours=cols[:2])
ax[1].set_title("Original reflectance")
refl.plot_spectra(ax=ax[1], indices=samples[:2], colours=cols[:2])
fig.show()

In this instance the ELC worked quite well using the panels placed 'on the target' such that they could properly account for path radiance (which is intentionally quite large in this example). However if we calculate and plot the residuals we can see that a significant topographic component remains.

In [ ]:
resid = np.mean( np.abs( reflectance.data / refl.data), axis=-1)
plt.figure(figsize=(10,5))
cm = plt.imshow(resid.T, cmap='coolwarm', vmin=0, vmax=2)
plt.xticks([])
plt.yticks([])
plt.title("Topographic residual")
plt.colorbar(cm, shrink=0.5)
plt.show()

If we use the panels that were close to the sensor (it is not uncommon that it is impossible to place panels on the target due to distance or access constraints), then the ELC completely fails and results in unusable spectra.

In [ ]:
from hylite.correct.illumination import ELC
elc = ELC([R25_close, R50_close]) # N.B we exclude the shaded panel from this as it would screw everything up
fig,ax = elc.quick_plot(thresh=100, figsize=(7,5))
fig.show()

In [ ]:
reflectance = radiance.copy(data=True) # create copy of data
_ = elc.apply(reflectance) # apply ELC

fig,ax = plt.subplots(1,2, figsize=(18,5))
ax[0].set_title("ELC corrected reflectance")
reflectance.plot_spectra(ax=ax[0], indices=samples[:2], colours=cols[:2])
ax[1].set_title("Original reflectance")
refl.plot_spectra(ax=ax[1], indices=samples[:2], colours=cols[:2])
fig.show()

In [ ]:
resid = np.mean( np.abs( reflectance.data / refl.data), axis=-1)
plt.figure(figsize=(10,5))
cm = plt.imshow(resid.T, cmap='coolwarm', vmin=0, vmax=2)
plt.xticks([])
plt.yticks([])
plt.title("Topographic residual")
plt.colorbar(cm, shrink=0.5)
plt.show()

In these cases, it is clearly necessary to take a different approach. In the following section, we use the panel spectra and the illumination model shown in Eq. 1 to empirically estimate the skylight, sunlight and path-radiance spectra from the panels to derive a more robust illumination correction.

### 4. Back-calculated illumination

We back calculate the illumination spectra by solving a linear system constructed that gives the measured radiance for each panel from their known reflectance values and unknown illumination spectra. Please refer to [Thiele et al., 2022](https://www.mdpi.com/2072-4292/14/1/5) for further details.

In [ ]:
# construct observations vector
B = np.array([ R25_close.get_mean_radiance(), R50_close.get_mean_radiance(), R99_shade.get_mean_radiance()] )

# construct linear system using known reflectance as coefficients
A = np.zeros((3,3,radiance.band_count()))
A[0,0] = R25_close.get_reflectance() * R25_close.skyview # skylight component of black panel
A[0,1] = R25_close.get_reflectance() * R25_close.alpha # sunlight component of black panel
A[0,2] = 30. # distance to panel (to resolve path radiance per meter)
A[1,0] = R50_close.get_reflectance() * R50_close.skyview # skylight component of black panel
A[1,1] = R50_close.get_reflectance() * R50_close.alpha # sunlight component of black panel
A[1,2] = 30. # distance to panel (to resolve path radiance per meter)
A[2,0] = R99_shade.get_reflectance() * R99_shade.skyview # skylight component of shaded panel
A[2,1] = 0 # reflected component is zero here
A[2,2] = 30. # distance to panel (to resolve path radiance per meter)

# solve it with numpy
skylight,sunlight,pathrad = np.linalg.solve( np.transpose(A, (2,0,1)), B.T ).T

In [ ]:
# make pretty plot
fig, ax = plt.subplots(1,2,figsize=(10,4),sharex=True, sharey=True)
ax = ax.ravel()

x = np.array(radiance.get_wavelengths())

ax[0].set_title("a. Flight A (panel spectra)")
ax[0].plot( x, R50_close.get_mean_radiance(), color='gray', lw=2, label='gray panel (50%)') # gray panel mean spectra
ax[0].plot( x, R25_close.get_mean_radiance(), color='black', lw=2, label='black panel (25%)') # black panel mean spectra
ax[0].plot( x, R99_shade.get_mean_radiance(), color='blue', lw=1, linestyle='--', label='shaded panel (90%)' ) # shaded panel
ax[0].set_ylabel(r"Radiance ($\frac{W}{nm \cdot m^{2} \cdot sr }$)")
ax[0].legend()

ax[1].set_title("d. Flight B (estimated illumination)")
ax[1].plot( x, sunlight, color='gold', lw=4, label='sunlight')
ax[1].plot( x, skylight, color='blue', lw=2, label='skylight')
ax[1].plot( x, pathrad*1000, color='gray', lw=1, linestyle='--', label='1km path radiance' )
ax[1].legend()

fig.tight_layout()
fig.show()

These spectra can then be combined with our geometric data to build a reflection model and thus correct the illumination effects across the scene.

In [ ]:
# multiply path radiance by per-pixel depth account for increasing radiance with depth
P_est = depth.copy(data=False)
P_est.data = depth.data * pathrad.data
P_est.set_wavelengths( radiance.get_wavelengths() )

# build illumination model using known geometry and estimated spectra
model = IlluModel( I = sunlight, # sunlight spectra
                   S = skylight, # skylight spectra
                   P = P_est, # path radiance
                   skv = skyview, # skyview factors
                   rf = ON, # reflectance factors
                   oc = None, # for this simple example we assume no projective shadows
                 )

# use it to convert radiance to estimated reflectance
reflectance = radiance.copy(data=False)
reflectance.data = model.getReflectance(radiance)

In [ ]:
fig,ax = plt.subplots(1,2, figsize=(18,5))
ax[0].set_title("Illumination corrected reflectance")
reflectance.plot_spectra(ax=ax[0], indices=samples[:2], colours=cols[:2])
ax[1].set_title("Original reflectance")
refl.plot_spectra(ax=ax[1], indices=samples[:2], colours=cols[:2])
fig.show()

In [ ]:
resid = np.mean( np.abs( reflectance.data / refl.data), axis=-1)
plt.figure(figsize=(10,5))
cm = plt.imshow(resid.T, cmap='coolwarm', vmin=0, vmax=2)
plt.xticks([])
plt.yticks([])
plt.title("Topographic residual")
plt.colorbar(cm, shrink=0.5)
plt.show()

Now, obviously we cheated in this example by using the same model to create our synthetic dataset as we used to correct it, but hopefully it serves as an illustrative example of how the methods in *hylite* can be used to extract information from calibration panels, estimate illumination spectra, combine them with geometric attributes to create an illumination model and, finally, derive corrected reflectance spectra.

Does it work on real data? Sometimes... just give it a try! 

¯\\_(ツ)_/¯

### 5. Statistical adjustments

In the real world, the illumination model (and resulting correcting) is always wrong. Hopefully it is a useful estimate that gets close to the actual reflectance, but often it doesn't.

Inspired by the `cfactor` algorithm for topographic correction, *hylite* includes a method for applying statistical adjustments to the correction factors. This calculates the correlation between modelled illumination and measured radiance, and then calculates an adjustment factor such that the regression line passes through the origin (as zero illumination should result in 0 at-target radiance). Like the original c-factor correction, this takes two forms; one which adds or removes path radiance until the regression passes through the origin, and the other that adds or removes downwelling light to achieve the same. 

First, lets build a simple illumination model that is roughly equivalent to the ELC and plot it.

In [ ]:
sunlight = R50_close.get_mean_radiance() / R50_close.get_reflectance() # will be approximately the sunlight
model = IlluModel( I = sunlight, # sunlight spectra
                   rf = ON, # reflectance factors
                 )

In [ ]:
# plot relationship between estimated downwelling illumination and measured reflected radiance
fig,ax = model.plot_fit(radiance, nb=1)
fig.show()

In [ ]:
# fit a cfactor correction to the data
model.fit(radiance, shift='y')
fig,ax = model.plot_fit(radiance, nb=1)
fig.show()

The fitting results are stored in the model object as either a radiance boost `model.rboost` (if fitted using `shift='y'`) or illumination `model.iboost` (if fitted using `shift='x'`). These adjustments will now be applied when using the model to estimate reflectance:

In [ ]:
# use it to convert radiance to estimated reflectance
reflectance = radiance.copy(data=False)
reflectance.data = model.getReflectance(radiance)

In [ ]:
fig,ax = plt.subplots(1,2, figsize=(18,5))
ax[0].set_title("Illumination corrected reflectance")
reflectance.plot_spectra(ax=ax[0], indices=samples[:2], colours=cols[:2])
ax[1].set_title("Original reflectance")
refl.plot_spectra(ax=ax[1], indices=samples[:2], colours=cols[:2])
fig.show()

In [ ]:
resid = np.mean( np.abs( reflectance.data / refl.data), axis=-1)
plt.figure(figsize=(10,5))
cm = plt.imshow(resid.T, cmap='coolwarm', vmin=0, vmax=2)
plt.xticks([])
plt.yticks([])
plt.title("Topographic residual")
plt.colorbar(cm, shrink=0.5)
plt.show()

These results are not great - an highlight the dangers of illumination corrections using regression based methods - but considering the illumination model we started with was very primitive (considering only estimated sunlight) it is still a big improvement (as compared to e.g. the ELC which also used the close panels). 

Hence, with real data it can be worth combining the illumination corrections descibed in Section 4 with these statistical adjustments; although the resulting spectra should be judged skeptically and ideally validated against in-situ methods.